In [2]:
import tensorflow as tf
from pathlib import Path
from tensorflow.contrib import predictor
import numpy as np

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [5]:
model_path = 'bert/export/1574008180'
predict_fn = predictor.from_saved_model(model_path)

W1117 17:32:21.603334 140272983353152 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/predictor/saved_model_predictor.py:153: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W1117 17:32:24.071917 140272983353152 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [7]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
model_path = 'bert/export/1574008180'
predict_fn_2 = predictor.from_saved_model(model_path)

In [9]:
predict_fn

SavedModelPredictor with feed tensors {'input_ids': <tf.Tensor 'input_ids_1:0' shape=(?, 384) dtype=int32>, 'input_mask': <tf.Tensor 'input_mask_1:0' shape=(?, 384) dtype=int32>, 'unique_ids': <tf.Tensor 'unique_ids_1:0' shape=(?,) dtype=int32>, 'segment_ids': <tf.Tensor 'segment_ids_1:0' shape=(?, 384) dtype=int32>} and fetch_tensors {'start_logits': <tf.Tensor 'unstack:0' shape=(?, 384) dtype=float32>, 'unique_ids': <tf.Tensor 'unique_ids_1:0' shape=(?,) dtype=int32>, 'end_logits': <tf.Tensor 'unstack:1' shape=(?, 384) dtype=float32>}

In [9]:
class BertQA(object):
    
    def __init__(self, model_path):
        #gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
        #self.session = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options),graph=tf.Graph())
        self.predict_fn = predictor.from_saved_model(model_path)
        
    def predict(self, data):
        return self.predict_fn(data)

In [10]:
model_path = 'bert/export/1574008180'
qa_model = BertQA(model_path)

W1117 17:16:21.187001 140348019226432 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/predictor/saved_model_predictor.py:153: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W1117 17:16:23.797347 140348019226432 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [21]:
unique_ids = [1020203]*1000
segment_ids =  np.array([[0]*384]*1000).reshape(1000,384)
input_ids = np.array([[0]*384]*1000).reshape(1000,384)
input_mask = np.array([[0]*384]*1000).reshape(1000,384)

In [22]:
predictions = qa_model.predict({'unique_ids': unique_ids, 'segment_ids': segment_ids, 'input_ids':input_ids, 'input_mask':input_mask})

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[1000,16,384,384] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node bert/encoder/layer_0/attention/self/Softmax (defined at <ipython-input-9-adb3c61cef82>:6) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[unstack/_809]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[1000,16,384,384] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node bert/encoder/layer_0/attention/self/Softmax (defined at <ipython-input-9-adb3c61cef82>:6) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor

In [6]:
?predict_fn